# Iterator e iterable

[iterador](https://docs.python.org/dev/glossary.html#term-iterator):

>An object representing a stream of data. Repeated calls to the iterator’s `__next__()` method (or passing it to the built-in function `next()`) return successive items in the stream. When no more data are available a `StopIteration` exception is raised instead. At this point, the iterator object is exhausted and any further calls to its `__next__()` method just raise `StopIteration` again. Iterators are required to have an `__iter__()` method that returns the iterator object itself so every iterator is also iterable and may be used in most places where other iterables are accepted. One notable exception is code which attempts multiple iteration passes. A container object (such as a list) produces a fresh new iterator each time you pass it to the `iter()` function or use it in a for loop. Attempting this with an iterator will just return the same exhausted iterator object used in the previous iteration pass, making it appear like an empty container.

[iterable](https://docs.python.org/dev/glossary.html#term-iterable):

> An object capable of returning its members one at a time. Examples of iterables include all sequence types (such as `list`, `str`, and `tuple`) and some non-sequence types like `dict`, file objects, and objects of any classes you define with an `__iter__()` method or with a `__getitem__()` method that implements Sequence semantics.

>Iterables can be used in a for loop and in many other places where a sequence is needed (`zip()`, `map()`, …). When an iterable object is passed as an argument to the built-in function `iter()`, it returns an iterator for the object. This iterator is good for one pass over the set of values. When using iterables, it is usually not necessary to call `iter()` or deal with iterator objects yourself. The for statement does that automatically for you, creating a temporary unnamed variable to hold the iterator for the duration of the loop. See also iterator, sequence, and generator.

Finalmente, la diferencia entre los dos es un poco sutil, por lo que vale la pena leer [una referencia más](https://www.geeksforgeeks.org/python-difference-iterable-iterator/)

> Note that every iterator is also an iterable, but not every iterable is an iterator. For example, a `list` is iterable but a `list` is not an iterator. An iterator can be created from an iterable by using the function `iter()`. To make this possible, the class of an object needs either a method `__iter__`, which returns an iterator, or a `__getitem__` method with sequential indexes starting with 0.

Seguramente leer esas referencias no es suficiente para tener un conocimiento útil del tema, que nos permita sacarle provecho. A continuación miraremos algunos ejemplos.

Empecemos con los *iterables*, la descripción más sencilla es que *son objetos que pueden retornar sus elementos uno a la vez*, lo que básicamente se refiere a que los podemos utilizar en un ciclo `for`. 

In [2]:
obj = [2,3,5,7]
for i in obj:
    print(i**2)

4
9
25
49


In [3]:
for c in "A long text":
    print(c)

A
 
l
o
n
g
 
t
e
x
t


Seguramente recuerdas que los objetos tipo `dict`, `tuple` y `set` también son iterables, es decir, se pueden usar en un ciclo `for`. Ahora lo miraremos en clases creadas por nosotros mismos. 

Considere una clase que guarda una lista de estudiantes

In [6]:
class StudentNames:
    def __init__(self, names):
        self._names = tuple(names)

Hasta ahora, los objetos de esta clase no podrían ser iterados:

In [7]:
names = StudentNames(["Sebastian", "Pepito", "Pablito"])
for n in names:
    print(n)

TypeError: 'StudentNames' object is not iterable

Existe una forma para poder iterar sobre los nombres, la cual consiste en acceder al atributo `StudentNames._names`

In [11]:
for n in names._names:
    print(n)

Sebastian
Pepito
Pablito


Sin embargo, a veces no queremos que los usarios acceda a algunos atributos de la clase, de hecho, la convención es que los atributos que empiezan con `_` no son usados directamente. Recordemos que en programación muchas veces nos interesa más la abstracción que la representación interna. En este caso, nos gustaría que alguien pudiera iterar sobre la lista de nombres sin decirle que lo tenemos guardado de alguna manera; para ese alguien no importa si cada nombre es consultado en una base de datos o generado aleatoriamente, por ejemplo.

Para que sea un *iterable* debe implementar el método `__iter__`, el cual retorna un *iterator*. Incluso si aún no sabemos qué es un *iterator*, sabemos que podemos obtener uno si le pasamos un *iterable* a la función `iter`

In [9]:
class StudentNames:
    def __init__(self, names):
        self._names = tuple(names)
    
    def __iter__(self):
        return iter(self._names)

In [10]:
names = StudentNames(["Sebastian", "Pepito", "Pablito"])
for n in names:
    print(n)

Sebastian
Pepito
Pablito


Ahora ya podemos construir objetos que pueden usarse en un ciclo for, lo cual hace que nuestro código sea un poco más legible y nos permite exponer únicamente lo que nos interesa exponer. Observa que nuestro objeto no será un *iterator*

In [12]:
next(names)

TypeError: 'StudentNames' object is not an iterator

Observemos que este mismo error nos sale si tratamos de usar `next` con otros del tipo *iterable* que no son *iterator*

In [22]:
next([12,3])

TypeError: 'list' object is not an iterator

In [23]:
next("cierto string")

TypeError: 'str' object is not an iterator

Ahora vamos a escribir la clase como un *iterator*, que a su vez es un *iterable*, pero se require que el método `__next__` se encuentre implementado.

In [15]:
class StudentNames:
    def __init__(self, names):
        self._names = tuple(names)
        self._idx = -1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._idx == len(self._names) - 1:
            raise StopIteration
        self._idx += 1
        return self._names[self._idx]

In [16]:
names = StudentNames(["Sebastian", "Pepito", "Pablito"])
for n in names:
    print(n)

Sebastian
Pepito
Pablito


Ahora mira qué pasa si usamos `next`

In [17]:
next(names)

StopIteration: 

Dado que ya habíamos iterado por todos los elementos, no podemos obtener un elemento siguiente. Ahora miremos otra cosa que podemos hacer si usamos iterators

In [18]:
names = StudentNames(["Sebastian", "Pepito", "Pablito"])
next(names)

'Sebastian'

In [19]:
next(names)

'Pepito'

In [20]:
next(names)

'Pablito'

In [21]:
next(names)

StopIteration: 

Antes de explorar lo que nos ofrece el módulo `itertools`, el objetivo principal de este notebook, nos falta entender otro elemento fundamental del arsenal de herramientas de Python: los generadores. 

[Generator](https://docs.python.org/3/glossary.html#term-generator)

> A function which returns a generator iterator. It looks like a normal function except that it contains yield expressions for producing a series of values usable in a for-loop or that can be retrieved one at a time with the next() function.

> Usually refers to a generator function, but may refer to a generator iterator in some contexts. In cases where the intended meaning isn’t clear, using the full terms avoids ambiguity.

[Generator iterator](https://docs.python.org/3/glossary.html#term-generator-iterator)

> An object created by a generator function.

> Each yield temporarily suspends processing, remembering the location execution state (including local variables and pending try-statements). When the generator iterator resumes, it picks up where it left off (in contrast to functions which start fresh on every invocation).

[Generator expression](https://docs.python.org/3/glossary.html#term-generator-expression)

> An expression that returns an iterator. It looks like a normal expression followed by a for clause defining a loop variable, range, and an optional if clause. The combined expression generates values for an enclosing function.

Ahora vamos a explorar unos ejemplos tomados de la [documentación oficial](https://docs.python.org/3/tutorial/classes.html#generators). 

La siguiente función recive una secuencia de datos y nos permite iterar en orden inverso

In [24]:
def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]

In [25]:
for c in reverse("Python"):
    print(c)

n
o
h
t
y
P


Observemos que esta no es una función como las que habíamos visto antes, en lugar de tener un `return` (o nada), tiene un `yield`. Miremos qué pasa si tratamos de usarla como una función cualquiera.

In [31]:
obj = reverse("The best class ever")
obj

<generator object reverse at 0x7fcd17803450>

podemos ver que esta función retorna un `generator`, que básicamente se comporta igual que un `iterator`

In [32]:
next(obj), next(obj)

('r', 'e')

In [33]:
for c in obj:
    print(c)

v
e
 
s
s
a
l
c
 
t
s
e
b
 
e
h
T


In [41]:
next(obj)

StopIteration: 

Básicamente los generadores nos permiten interrumpir la ejecución de código y retomarla en otro momento (esta idea es bastante importante, ya que será usada cuando se estudie programación asíncrona). 

Finalmente, un *generator expression*  es una forma compacta de crear generadores simples:

In [38]:
squares = (i**2 for i in range(10))
squares

<generator object <genexpr> at 0x7fcd165b2a50>

In [39]:
next(squares), next(squares), next(squares)

(0, 1, 4)

In [40]:
for s in squares:
    print(s)

9
16
25
36
49
64
81


In [42]:
next(squares)

StopIteration: 

## Itertools

La [documentación oficial](https://docs.python.org/3/library/itertools.html) dice

>Functions creating iterators for efficient looping

y también:

> The module standardizes a core set of fast, memory efficient tools that are useful by themselves or in combination. Together, they form an “iterator algebra” making it possible to construct specialized tools succinctly and efficiently in pure Python.

Lo que básicamente quiere decir que contiene funciones que crean iteradores y operan sobre iteradores. Usualmente, usar iteradores y el módulo `itertools` nos permite escribir código limpio y eficiente.

Honestamente, cuando pensaba escribir esta sección encontré un [artículo genial](https://realpython.com/python-itertools/) sobre itertools, ya tienes todo lo necesario para entenderlo, así que puedes leerlo por tu cuenta. Al ver ese artículo me pareció innecesario reescribir acá algo similar.